In [1]:
# pip install gradio 
# !pip install vllm

In [1]:
from langchain.llms import CTransformers
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch
from langchain_community.llms import VLLM


In [2]:
from langchain.llms import CTransformers
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


# model_name="/home-old/s223795137/models/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa",  

model_name='microsoft/phi-4'

# model_name  = 'meta-llama/Llama-3.2-3B-Instruct'

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024,device='cuda')

In [3]:
from langchain_community.llms import VLLM

llm = VLLM(
    model=model_name,
    tensor_parallel_size=1,
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=32000,
)


INFO 02-25 13:42:08 config.py:510] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
INFO 02-25 13:42:08 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/phi-4, num_scheduler_s

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 02-25 13:42:22 model_runner.py:1099] Loading model weights took 27.3875 GB
INFO 02-25 13:42:24 worker.py:241] Memory profiling takes 1.34 seconds
INFO 02-25 13:42:24 worker.py:241] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.90) = 71.20GiB
INFO 02-25 13:42:24 worker.py:241] model weights take 27.39GiB; non_torch_memory takes 0.17GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 41.50GiB.
INFO 02-25 13:42:24 gpu_executor.py:76] # GPU blocks: 13598, # CPU blocks: 1310
INFO 02-25 13:42:24 gpu_executor.py:80] Maximum concurrency for 16384 tokens per request: 13.28x
INFO 02-25 13:42:26 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.45it/s]

INFO 02-25 13:42:40 model_runner.py:1535] Graph capturing finished in 14 secs, took 1.07 GiB
INFO 02-25 13:42:40 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 18.01 seconds


In [4]:
# llm = HuggingFacePipeline(pipeline=pipe)

In [4]:
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
        self.batch_size = batch_size  # Control batch size

    def embed_documents(self, texts):
        texts = ["search_document: "+i for i in texts]
        return self.model.encode(
            texts, 
            convert_to_numpy=True, 
            device=self.device, 
            batch_size=self.batch_size
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            ['search_query: '+text], 
            convert_to_numpy=True, 
            device=self.device
        )[0].tolist()


# Initialize the embedding model
local_embeddings = SentenceTransformerEmbeddings(batch_size=64)

!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [5]:
from langchain.vectorstores import Chroma

# from langchain.retrievers import BaseRetriever

# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings , persist_directory="./chroma_db_pages_v3_2.5k")

# Load your Chroma vector store

vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)

# Create a retriever from the vector store

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Fetch top 5 chunks

/tmp/ipykernel_620564/2829555206.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)


In [7]:
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain

# Define your prompt template

prompt_template = PromptTemplate(

    input_variables=["context", "question"],
    template="""<|im_start|>system<|im_sep|><|im_end|>

<|im_start|>user<|im_sep|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|im_start|>assistant<|im_sep|>""",
 
)

# Define your LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM


stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  # This should match the input variable in your prompt
)
# Combine into a RetrievalQA chain


def extract_answers(answers):


    answers = answers.split("<|im_start|>assistant<|im_sep|>")[-1]
    # answers = answers.split("<|eot_id|>")[-1]

    return answers


# Function to query the RAG model
# def query_rag_model(query):
    
#     response = rag_chain.run(query)

#     response = extract_answers(response)
    
#     return response

/tmp/ipykernel_620564/1921380948.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM
/tmp/ipykernel_620564/1921380948.py:19: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(


In [ ]:
# class ReversedStuffDocumentsChain(StuffDocumentsChain):
#     def _call(self, inputs: dict) -> dict:
#         # Debugging: Check the inputs dictionary structure
#         print(f"Inputs received in _call: {inputs}")

#         # Ensure that the "input_documents" key exists in inputs
#         if "input_documents" not in inputs:
#             raise KeyError("The key 'input_documents' was not found in inputs")

#         # Get the documents from the inputs (input_documents)
#         documents = inputs["input_documents"]
        
#         # Reverse the order of the documents
#         reversed_documents = documents[::-1]
        
#         # Pass the reversed documents back to the original StuffDocumentsChain logic
#         # Convert list of documents into a single string if necessary
#         inputs["input_documents"] = reversed_documents
        
#         # Pass the adjusted inputs to the parent class
#         return super()._call(inputs)
    
    
# llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM

# # Use the custom chain
# reversed_stuff_chain = ReversedStuffDocumentsChain(
#     llm_chain=llm_chain,
#     document_variable_name="context"  # This should match the input variable in your prompt
# )

In [ ]:
# # Update the retriever's 'k' value
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Use the dynamic 'k'

# rag_chain_reversed = RetrievalQA(

#     retriever=retriever,

#     combine_documents_chain=reversed_stuff_chain)

In [ ]:
# res =rag_chain_reversed.run("Suggest me a good password to use in Deakin. I love dogs.")
# res

Inputs received in _call: {'input_documents': [Document(metadata={'source': '/weka/s223795137/Crawl_data/pages_datasets/IT Help - Change your password using the Deakin website - httpshelp.deakin.edu.auithelphelp.deakin.edu.auithelpid=kb_article_view&sys_kb_id=8324c103c3420a14d95d79aa050131a6.txt'}, page_content='search_document: IT Help - Change your password using the Deakin website Skip to page content Welcome to Deakin, let\'s get started with your Digital Essentials! Get help with activating your Deakin IT account,  connecting to wifi (Eduroam), printing on campus, installing Microsoft 365 apps, and accessing software for your units. Click here to visit "Getting Started with your Digital Essentials". MORE Toggle navigation Log in Home Knowledge IT Help (Knowledge Base) Access & Passwords IT Help - Change your password using the Deakin website BMP Created with Sketch. BMP ZIP Created with Sketch. ZIP XLS Created with Sketch. XLS TXT Created with Sketch. TXT PPT Created with Sketch. 

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.47s/it, est. speed input: 780.89 toks/s, output: 75.32 toks/s]


'When creating a secure password, it\'s crucial to ensure it adheres to specific guidelines for strength and security. Based on the Deakin guidelines you provided, here are some key points to keep in mind:\n\n1. **Length**: The password should be between 15 to 30 characters.\n2. **Complexity**: It should include a mix of uppercase and lowercase letters, numbers, and symbols.\n3. **Uniqueness**: The password should be different from your previous five passwords.\n\nGiven that you love dogs, here\'s a strategy to create a good password while including your interest:\n\n1. Think of a phrase or saying about dogs that you’d remember easily.\n2. Incorporate numbers, symbols, and uppercase letters.\n3. Avoid common words or phrases that could be easily guessed.\n\nFor example:\n- Start with a phrase like "MyDogLovesToRunAndFetch"\n- Add numbers and symbols to enhance security: "MyD0gL0v3sT0Run&F3tch!"\n- Adjust the length to fit between 15 and 30 characters and ensure it meets the complexity 

In [ ]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Use the dynamic 'k'

# rag_chain = RetrievalQA(

#     retriever=retriever,

#     combine_documents_chain=stuff_chain)

In [12]:
res =query_rag_model("Suggest me a good password to use in Deakin. I love dogs.",k=5)
print(res)

/tmp/ipykernel_620564/2208830183.py:29: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(
/tmp/ipykernel_620564/2208830183.py:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)
Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.49s/it, est. speed input: 602.89 toks/s, output: 66.17 toks/s]

When choosing a password, it's important to prioritize security. Given Deakin's requirement that a password must be between 15 to 30 characters and not match any of the previous five passwords, here is a suggestion:

1. **Combine Personal Interests and Random Elements:**
   - **Base:** Start with something meaningful or memorable to you. Since you love dogs, use a breed name, dog-related phrase, or a memorable dog’s name.
   - **Random Characters:** Add random numbers and special characters to increase complexity and security.

2. **Example Password:**
   - Let's say your favorite dog breed is "Labrador," and you also love hiking. A possible password could be: `Labr@d0r!L0v3sH!k1ngHills987!`
   - **Breakdown:** 
     - "Labr@d0r" incorporates a breed name with a mix of upper and lowercase letters and special characters.
     - "!L0v3sH!k1ngHills987!" adds more complexity.
     - The length is well within Deakin's requirements.

3. **Security Tips:**
   - Avoid using easily guessed info

In [13]:
import gradio as gr

# Define the function that Gradio will call

# def rag_interface(query):
#     response = query_rag_model(query)
#     return response



# iface = gr.Interface(
#     fn=rag_interface,  # Function to call
#     inputs="text",     # Input type (text box)
#     outputs="text",    # Output type (text box)
#     title="LangChain RAG Model",  # Title of the interface
#     description="Enter your query and get answers from the LangChain RAG model."
# )

# # Launch the interface



def query_rag_model(query, k):

    
    # Update the retriever's 'k' value
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})  # Use the dynamic 'k'

    rag_chain = RetrievalQA(

        retriever=retriever,

        combine_documents_chain=stuff_chain)
    
    
    # Run the RAG pipeline
    response = rag_chain.run(query)
    
    # Extract the answer
    response = extract_answers(response)
    
    return response

# Gradio interface with a Slider for 'k'
def rag_interface(query, k=5):
    response = query_rag_model(query, k)
    return response

# Set up the Gradio interface with a Slider for 'k'
iface = gr.Interface(
    fn=rag_interface,   # Function to call
    inputs=[            # List of inputs: query and k (slider)
        gr.Textbox(label="Query", placeholder="Enter your question here..."),
        gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Documents to Retrieve (k)")
    ], 
    outputs="text",     # Output type (text box)
    title="LangChain RAG Model",
    description="Enter your query and adjust 'k' to control the number of documents to retrieve for answering."
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://9d70abdda4264669b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it, est. speed input: 2417.03 toks/s, output: 69.53 toks/s]


In [10]:
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a7c61e2a784f5484ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
%%time
query_rag_model('Can I install a torrent client in my Deakin PC?',5)

/tmp/ipykernel_1565481/3741844905.py:29: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(
/tmp/ipykernel_1565481/3741844905.py:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)
Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it, est. speed input: 710.34 toks/s, output: 75.29 toks/s]

CPU times: user 3.88 s, sys: 46.2 ms, total: 3.92 s
Wall time: 3.96 s


"When using your Deakin-managed PC, it is important to comply with university policies related to software installation and usage. While Deakin provides tools like the Software Center on Digital Services-managed computers, installing torrent clients is generally discouraged for the following reasons:\n\n1. **Policy and Security Concerns**: Deakin University has specific IT policies to ensure the security and integrity of its network and computers. Torrent clients can pose security risks and may be used for distributing and downloading unauthorized content, which violates university policy and could lead to security breaches.\n\n2. **Network Usage**: The university network is optimized for academic and professional use. Torrenting can consume significant bandwidth, affecting the performance of the network for other users.\n\n3. **Software Guidelines**: The Software Center and IT guidelines typically focus on providing access to applications that support academic work and university oper

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.75s/it, est. speed input: 577.34 toks/s, output: 75.07 toks/s]


In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load all text files from a folder
loader = DirectoryLoader("/weka/s223795137/Crawl_data/pages_datasets", glob="*.txt")  
documents = loader.load()


In [4]:
li_len = []
for i in range(len(documents)):
    li_len.append(len(documents[i].page_content))

In [12]:
import matplotlib.pyplot as plt


plt.hist(li_len)

NameError: name 'li_len' is not defined

/bin/bash: line 1: nividia-smi: command not found


## DRAFT

In [8]:
from sentence_transformers import SentenceTransformer
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
import torch
from langchain import LLMChain, PromptTemplate
from langchain.chains import RetrievalQA, StuffDocumentsChain
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.vectorstores import Chroma
import gradio as gr
import json
import os
from langchain_community.llms import VLLM




# model_name="/home-old/s223795137/models/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/e1945c40cd546c78e41f1151f4db032b271faeaa",  

model_name='microsoft/phi-4'

# model_name  = 'meta-llama/Llama-3.2-3B-Instruct'

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=32000,device='cuda')




# llm = HuggingFacePipeline(pipeline=pipe)


llm = VLLM(
    model=model_name,
    tensor_parallel_size=1,
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=32000,
)


class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name="nomic-ai/nomic-embed-text-v1.5", device=None, batch_size=32):
        super().__init__()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=self.device)
        self.batch_size = batch_size  # Control batch size

    def embed_documents(self, texts):
        texts = ["search_document: "+i for i in texts]
        return self.model.encode(
            texts, 
            convert_to_numpy=True, 
            device=self.device, 
            batch_size=self.batch_size
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            ['search_query: '+text], 
            convert_to_numpy=True, 
            device=self.device
        )[0].tolist()


local_embeddings = SentenceTransformerEmbeddings(batch_size=64)





vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)


# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Fetch top 5 chunks





# Define your prompt template

prompt_template = PromptTemplate(

    input_variables=["context", "question"],
    template="""<|im_start|>system<|im_sep|><|im_end|>

<|im_start|>user<|im_sep|>Answer the question based on the context below:\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:<|im_start|>assistant<|im_sep|>""",
 
)

# Define your LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM


stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  
)


def extract_answers(answers):


    answers = answers.split("<|im_start|>assistant<|im_sep|>")[-1]

    return answers




import datetime

chat_history_file = "chat_history.json"



def log_interaction(query, response, filename="chat_history.json"):
    """Logs the user query and model response to a JSON file."""
    
    # Create an empty list if file doesn't exist
    if not os.path.exists(filename):
        history = []
    else:
        with open(filename, "r", encoding="utf-8") as file:
            try:
                history = json.load(file)  # Load existing history
            except json.JSONDecodeError:
                history = []  # If file is empty or corrupted, reset it

    # New entry
    entry = {
        "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "user_query": query,
        "model_response": response
    }

    # Append new entry and save back to JSON file
    history.append(entry)

    with open(filename, "w", encoding="utf-8") as file:
        json.dump(history, file, ensure_ascii=False, indent=4)




def query_rag_model(query, k):
    # Update the retriever's 'k' value
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})

    rag_chain = RetrievalQA(
        retriever=retriever,
        combine_documents_chain=stuff_chain
    )
    
    # Run the RAG pipeline
    response = rag_chain.run(query)
    
    # Extract the answer
    response = extract_answers(response)

    # Log interaction in JSON format
    log_interaction(query, response)

    return response


# def store_rating(query, response, rating):
#     """Stores the rating after the response is shown."""
    
#     log_interaction(query, response, rating)
#     return "Your feedback has been recorded! ✅"


# Gradio interface with a Slider for 'k'
def rag_interface(query, k=5):
    response = query_rag_model(query, k)
    return response


# with gr.Blocks() as iface:
    
#     gr.Markdown("# 📚 Deakin IT AI Help Desk")

#     query_input = gr.Textbox(label="Query", placeholder="Enter your question here...")
#     k_slider = gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Documents to Retrieve (k)")
#     submit_btn = gr.Button("Get Response")

#     response_output = gr.Textbox(label="Model Response", interactive=False)
    
#     # Feedback Section (Only appears after response)
#     rating_dropdown = gr.Dropdown(choices=["N/A", "1", "2", "3", "4", "5"], value="N/A", label="Rate the Response (Optional)")
#     feedback_btn = gr.Button("Submit Rating")

#     feedback_message = gr.Textbox(label="Feedback Status", interactive=False)

#     # Step 1: Get Response
#     submit_btn.click(fn=query_rag_model, inputs=[query_input, k_slider], outputs=response_output)

#     # Step 2: Submit Rating
#     feedback_btn.click(fn=store_rating, inputs=[query_input, response_output, rating_dropdown], outputs=feedback_message)


# Set up the Gradio interface with a Slider for 'k'
iface = gr.Interface(
    # gr.Markdown("# 📚 Deakin IT AI Help Desk")
    
    fn=rag_interface,   # Function to call
    inputs=[            # List of inputs: query and k (slider)
        gr.Textbox(label="Query", placeholder="Enter your question here..."),
        gr.Slider(minimum=1, maximum=25, step=1, value=5, label="Number of Documents to Retrieve (k)"),


    ], 
    outputs="text",     # Output type (text box)
    title="# 📚 Deakin IT AI Help Desk",
    description="Enter your query and adjust 'k' to control the number of documents to retrieve for answering."
)




iface.launch(share=True)


/home/s223795137/.conda/envs/llava/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


INFO 02-25 14:01:09 config.py:510] This model supports multiple tasks: {'score', 'generate', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 02-25 14:01:09 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=microsoft/phi-4, num_scheduler_s

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 02-25 14:01:19 model_runner.py:1099] Loading model weights took 27.3875 GB
INFO 02-25 14:01:20 worker.py:241] Memory profiling takes 1.33 seconds
INFO 02-25 14:01:20 worker.py:241] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.90) = 71.20GiB
INFO 02-25 14:01:20 worker.py:241] model weights take 27.39GiB; non_torch_memory takes 0.17GiB; PyTorch activation peak memory takes 2.14GiB; the rest of the memory reserved for KV Cache is 41.50GiB.
INFO 02-25 14:01:20 gpu_executor.py:76] # GPU blocks: 13598, # CPU blocks: 1310
INFO 02-25 14:01:20 gpu_executor.py:80] Maximum concurrency for 16384 tokens per request: 13.28x
INFO 02-25 14:01:22 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.06it/s]

INFO 02-25 14:01:39 model_runner.py:1535] Graph capturing finished in 17 secs, took 1.07 GiB
INFO 02-25 14:01:39 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 20.65 seconds



!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>
/tmp/ipykernel_623241/3218999341.py:73: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./chroma_db_pages_v3_2.5k", embedding_function=local_embeddings)
/tmp/ipykernel_623241/3218999341.py:94: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)  # Replace with your LLM
/tmp/ipykernel_623241/3218999341.py:97: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_docum

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7b7a86eb284b0681cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipykernel_623241/3218999341.py:152: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(
/tmp/ipykernel_623241/3218999341.py:158: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)
Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.23s/it, est. speed input: 620.25 toks/s, output: 75.49 toks/s]
